In [1]:
import json
import mailbox
import email

In [2]:
fn='messages/inbox/anna_d0ta1k5dmg/message_1.json'
data = json.load(open(fn))

In [3]:
def utfdemangle(s):
    # FB delivers strange JSON where all strings are encoded in UTF8 bytes
    return s.encode('latin1').decode('utf8')

In [4]:
msg=data['messages'][2]

In [5]:
msg

{'sender_name': 'Christian Gollwitzer',
 'timestamp_ms': 1651301847108,
 'content': 'Bin leider etwas spÃ¤ter dran, gab mal wieder ZugausfÃ¤lle. Bin wahrscheinlich erst um 9:30 da.\n\nBin leider etwas spÃ¤ter dran, gab mal wieder ZugausfÃ¤lle. Bin wahrscheinlich erst um 9:30 da.',
 'type': 'Generic',
 'is_unsent': False,
 'is_taken_down': False,
 'bumped_message_metadata': {'bumped_message': 'Bin leider etwas spÃ¤ter dran, gab mal wieder ZugausfÃ¤lle. Bin wahrscheinlich erst um 9:30 da.\n\nBin leider etwas spÃ¤ter dran, gab mal wieder ZugausfÃ¤lle. Bin wahrscheinlich erst um 9:30 da.',
  'is_bumped': False}}

In [6]:
m=email.message.EmailMessage()


In [7]:
m['From']=utfdemangle(msg['sender_name'])
m['To']='Group'
m.set_content(utfdemangle(msg['content']))


In [8]:
print(str(m))

From: Christian Gollwitzer
To: Group
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: quoted-printable
MIME-Version: 1.0

Bin leider etwas sp=C3=A4ter dran, gab mal wieder Zugausf=C3=A4lle. Bin wahrs=
cheinlich erst um 9:30 da.

Bin leider etwas sp=C3=A4ter dran, gab mal wieder Zugausf=C3=A4lle. Bin wahrs=
cheinlich erst um 9:30 da.

